In [ ]:
import os
import sys
import copy
import torch
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
sys.path.append("/home/zanardi/Codes/ML/ROMAr/romar/")
from romar import env
from romar.const import *
from romar import backend as bkd
from romar.systems import BoxAd

In [ ]:
env_opts = {
  "backend": "numpy",
  "device": "cpu",
  "device_idx": 0,
  "nb_threads": 2,
  "epsilon": None,
  "floatx": "float64"
}
env.set(**env_opts)

In [ ]:
path_to_dtb = "/home/zanardi/Codes/ML/ROMAr/romar/examples/database"
out_path = "/home/zanardi/Codes/ML/ROMAr/romar/examples/database/utils/rom_data/"
os.makedirs(out_path, exist_ok=True)

In [ ]:
model = BoxAd(
  species={k: path_to_dtb + "/species/" + k + ".json" for k in ("Ar", "Arp", "em")},
  kin_dtb=path_to_dtb + "/rates/kin_fit.p",
  rad_dtb=path_to_dtb + "/rates/rad_fit.p",
  use_rad=True,
  use_tables=False
)

In [ ]:
sol_ref = "/home/zanardi/Workspace/AirDatabase/ThermoChemModels/StS/argon/testcase/0d_box_ad/output_box/"

In [ ]:
columns = "t X_em X_Ar X_Arp Th Te rho p n E".split()
box = np.loadtxt(sol_ref + "/box.dat")
box = pd.DataFrame(box, columns=columns)
box

In [ ]:
t = box["t"].values
rho, Th0, Te0 = [box[k][0] for k in ("rho", "Th", "Te")]

In [ ]:
def read_pop(name):
  ni_gi  = np.loadtxt(sol_ref + f"/pop_{name}.dat", comments="&", skiprows=2)[:,1]
  levels = model.mix.species[name].lev
  nb_pts = len(ni_gi) / len(levels["g"])
  ni_gi  = np.stack(np.split(ni_gi, nb_pts, axis=0), axis=0)
  return ni_gi * levels["g"].numpy()

In [ ]:
n = {k: read_pop(k) for k in ("Ar", "Arp")}
n["em"] = (box["X_em"] * box["n"]).values.reshape(-1,1)
n_true = np.hstack([n[k] for k in model.species_order]).T
rho_true = model.mix.get_rho(bkd.to_torch(n_true)).numpy()
T_true = box[["Th", "Te"]].values.T
rho = np.sum(rho_true[:,0])
# rho = 1e-2

In [ ]:
# y_true = np.vstack([rho_true/rho, T_true]).T
# np.savetxt(out_path + "/y.txt", y_true)
# np.savetxt(out_path + "/t.txt", t)

In [ ]:
ei = model.mix.species["em"].indices
y0 = np.zeros(model.nb_eqs)
y0[:model.mix.nb_comp] = rho_true[:,0] / rho
y0[-2] = Th0
y0[-1] = model.mix.get_pe(Te=Te0, ne=n_true[ei,0])
y0

In [ ]:
# rho = np.loadtxt(out_path + "/rho.txt")
# y_true = np.loadtxt(out_path + "/y_plato.txt")

In [ ]:
# w0, Th0, Te0 = y_true[0,:-2], y_true[0,-2], y_true[0,-1]
# model.mix.set_rho(rho)
# n0 = model.mix.get_n(w0)
# pe0 = model.mix.get_pe(Te=Te0, ne=n0[0])

In [ ]:
# ei = model.mix.species["em"].indices
# y0 = np.zeros(model.nb_eqs)
# y0[:model.mix.nb_comp] = w0
# y0[-2] = Th0
# y0[-1] = pe0
# y0

In [ ]:
y, runtime = model.solve_fom(t, y0, rho, linear=False)
runtime

In [ ]:
n_pred, *T_pred = model.get_prim(y, clip=False)

In [ ]:
# y[-1] = T_pred[-1]
# np.savetxt(out_path + "/y_romar.txt", y.T)

In [ ]:
err = []
for i in range(2):
  ierr = np.mean(np.abs((T_true[i] - T_pred[i])/T_true[i]))
  err.append(ierr)
  plt.semilogx(t[1:], T_true[i][1:])
  plt.semilogx(t[1:], T_pred[i][1:], ls="--", color=plt.gca().lines[-1].get_color())
print("T")
print(np.mean(err))
plt.show()
plt.close()

In [ ]:
for s in model.mix.species.values():
  print(s.name)
  err = []
  for i in s.indices:
    ierr = np.mean(np.abs((n_true[i] - n_pred[i])/n_true[i]))
    err.append(ierr)
    plt.loglog(t[1:], n_true[i][1:])
    plt.loglog(t[1:], n_pred[i][1:], ls="--", color=plt.gca().lines[-1].get_color())
  print(np.mean(err))
  plt.show()
  plt.close()

In [ ]:
# Plot ni solution
for i in range(model.nb_comp):
  plt.loglog(t[1:], n_true[i][1:])
  plt.loglog(t[1:], n_pred[i][1:], ls="--", lw=2.0, color=plt.gca().lines[-1].get_color())
plt.xlabel("$t$ [s]")
plt.ylabel("$n_i$ [m$^{-3}$]")
plt.savefig(out_path + "/ni_sol.png")
plt.close()

# Plot ni error
err = 100 * np.abs((n_true - n_pred)/n_true)
plt.loglog(t[1:], err.T[1:])
plt.xlabel("$t$ [s]")
plt.ylabel("$n_i$ error [%]")
plt.savefig(out_path + "/ni_err.png")
plt.close()

In [ ]:
from tqdm import tqdm

In [ ]:
%timeit model._jac(0.0, yi)

In [ ]:
%timeit sp.optimize.approx_fprime(xk=yi, f=lambda z: model._fun_np(0.0, z), epsilon=1e-15)

In [ ]:
err1, err2 = [], []
for yi in tqdm(y.T[::8]):
  jp = model._jac(0.0, yi)
  # Finite difference Jacobian
  jt = sp.optimize.approx_fprime(
    xk=yi,
    f=lambda z: model._fun_np(0.0, z),
    epsilon=1e-15
  )
  err1.append(np.linalg.norm(jp-jt)/np.linalg.norm(jt))
  err2.append(np.linalg.norm(jp-jt)/np.linalg.norm(jp))
err1 = np.asarray(err1)
err2 = np.asarray(err2)

In [ ]:
plt.loglog(t[::8], err1)
plt.show()
plt.close()

In [ ]:
plt.loglog(t[::8], err2)
plt.show()
plt.close()

In [ ]:
stop

Linear model

In [ ]:
ylin, runtime = model.solve_fom(t, y0, rho, linear=True)
runtime

In [ ]:
n_lin, *T_lin = model.get_prim(ylin, clip=False)
nt_lin = n_lin.shape[1]

In [ ]:
err = []
for i in range(2):
  ierr = np.mean(np.abs((T_pred[i][:nt_lin] - T_lin[i])/T_pred[i][:nt_lin]))
  err.append(ierr)
  plt.semilogx(t[1:nt_lin], T_pred[i][1:nt_lin])
  plt.semilogx(t[1:nt_lin], T_lin[i][1:], ls="--", color=plt.gca().lines[-1].get_color())
print("T")
print(np.mean(err))
plt.show()
plt.close()

In [ ]:
for s in model.mix.species.values():
  print(s.name)
  err = []
  for i in s.indices:
    ierr = np.mean(np.abs((n_pred[i] - n_lin[i])/n_pred[i]))
    err.append(ierr)
    plt.loglog(t[1:], n_pred[i][1:])
    plt.loglog(t[1:], n_lin[i][1:], ls="--", color=plt.gca().lines[-1].get_color())
  print(np.mean(err))
  plt.show()
  plt.close()

Sample time points

In [ ]:
y_interp = sp.interpolate.interp1d(t, y, kind="linear", axis=-1)

In [ ]:
tvec = np.array([0.0] + np.geomspace(1e-12, 1e-4, 100).tolist())
tsamples = np.geomspace(1e-13, 1e-4, 10)

In [ ]:
for i, ti in enumerate(tsamples):
  
  print(f"Time {i+1}: {ti} s")
  print("="*40)

  yref = y_interp(tvec+ti)
  ylin, _ = model.solve_fom(tvec, yref[:,0], rho, linear=True)

  tmin = model.compute_timescale(yref[:,0], rho)
  tmax = model.compute_lin_tmax(tvec, yref, rho, err_max=25.0)
  print(tmin, tmax)

  for s in model.mix.species.values():
    for i in s.indices:
      plt.loglog(tvec[1:], yref[i][1:])
      plt.loglog(tvec[1:], ylin[i][1:], ls="--", color=plt.gca().lines[-1].get_color())
    plt.axvline(x=tmax, color='r', linestyle='--')
    plt.show()
    plt.close()